# Spanish 81: Ensayo 4
Para este proyecto, estoy analizando dos libros de cocina por Simone K. Ortega: *Nuevas Recetas de Cocina (1984)* y *1080 Recetas de Cocina, Alianza Editorial (1998)*. Quise estudiar el costo promedio de los ingredientes en una manera cuantitativa usando el apéndice de cada libro para estimar la cantidad y el tipo de ingredientes en ambos libros. Usé el sitio web de Carrefour, un supermercado españolo, para encontrar un precio por ingredientes. La manera en que hice esta analisis esta fue:
- captura y itemiza los elementos del apéndice de cada libro y cuéntalos
- encontra el precio por cada elemento de cada libro
- multiplica el precio por la frequencia por cada elemento y sumar cada entrada por cada libro. Las sumas resultantes representan una estimación del costo promedio.


## Itemiza el apendice

In [1]:
# imports
import re
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import random

In [2]:
## clean text into list of words ##
def process_text(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
        # Extracting only words (no numbers, special characters)
        words = re.findall(r'\b[A-Za-z]+\b', text)
        # Converting words to lowercase
        words = [word.lower() for word in words]
        return words

file_path = './NuevasAppendix.txt'  # Change this to your file path
words_to_remove = ['a', 'con', 'la', 'al', 'y', 'el', 'de', 'o', 'en']
word_list_nuevas = [word for word in process_text(file_path) if word not in words_to_remove]
word_list_nuevas = list(set(word_list_nuevas))
print(word_list_nuevas)
print(len(word_list_nuevas))

['galletas', 'para', 'paletilla', 'flamenca', 'otra', 'fideos', 'mariscos', 'pepinos', 'gur', 'kiwis', 'picada', 'calvados', 'cebollas', 'bu', 'espada', 'tona', 'cuartos', 'pescado', 'candy', 'sal', 'trufas', 'del', 'adobados', 'hervido', 'sopa', 'nabos', 'caldeirada', 'canela', 'alitas', 'boquerones', 'manzanas', 'crema', 'gambas', 'congelado', 'horchata', 'guisado', 'escabechadas', 'guisadas', 'rollitos', 'so', 'h', 'pasas', 'frescos', 'rehogado', 'rollo', 'tarta', 'pimienta', 'zanahorias', 'picante', 'cocido', 'frambuesas', 'pizza', 'verdes', 'rece', 'estofadas', 'revueltos', 'cubrir', 'ahumada', 'historiadas', 'pollitos', 'merengues', 'aguacate', 'andaluza', 'aleta', 'cocer', 'cebolla', 'agua', 'natillas', 'milanesa', 'queso', 'vinagre', 'champagne', 'ta', 'perdices', 'relleno', 'yemas', 'jugo', 'quesitos', 'acelgas', 'ter', 'madeira', 'salado', 'leche', 'espinacas', 'sin', 'coliflor', 'aceitunas', 'toma', 'mayonesa', 'pere', 'variadas', 'pintas', 'pet', 'aleman', 'patatas', 'cabel

In [3]:
## scrape prices from grocery store ##
# https://www.carrefour.es/ 
def scrape_all_text(url):
    time.sleep(random.random()*5+5)
    # Start a Chrome webdriver
    # Create Chrome options with incognito mode
    chrome_options = Options()
    chrome_options.add_argument('--incognito')

    driver = webdriver.Chrome(options=chrome_options)

    driver.get(url)  # Open the provided URL
    time.sleep(random.random()*2 + 4)
    try:
       # If there's a button with the text "Aceptar" (Accept), locate it and click
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Aceptar')]"))
        )
        accept_button.click()
        time.sleep(random.random()*2 + 6)
        # Get the full page source
        page_source = driver.page_source

        # Use BeautifulSoup to parse the page source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extract all visible text
        all_text = soup.get_text()

        # process text to isolate price
        # Find the index of € symbol
        lower_case = all_text.lower()
        index_of_euro = lower_case.find(search_word)


        # Check if € symbol is found
        if index_of_euro != -1:
            # Remove text before € symbol
            result_string = lower_case[index_of_euro + 1:]
            result_string = result_string[result_string.find("€"):]
            result_string = result_string[1:result_string.find(" ")]
            result_string = result_string.replace(",", ".")
            print(result_string)

    except:
        result_string = 0
        driver.quit()
    
    finally:
        driver.quit()
        try:
            return float(result_string)
        except:
            return 0

# Example usage:
url = 'https://www.carrefour.es/?q='  # Replace this with the URL of the website you want to scrape
search_word = 'cerveza'  # Replace this with the word you want to search for

search_result = scrape_all_text(url+search_word)


In [ ]:
import concurrent.futures

file_path = './NuevasAppendix.txt'  # Change this to your file path
words_to_remove = ['a', 'con', 'la', 'al', 'y', 'el', 'de', 'o', 'en']
word_list_nuevas = [word for word in process_text(file_path) if word not in words_to_remove]

def get_text_for_word(word):
    return scrape_all_text(url + word)

# Using ThreadPoolExecutor to limit the number of active threads to 10
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit tasks to the thread pool and get futures
    futures = [executor.submit(get_text_for_word, word) for word in word_list_nuevas]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)

    # Retrieve results
    prices_nuevas = [future.result() for future in futures]

print(prices_nuevas)

In [ ]:
## get sum ##
nuevas_no_zero = [value for value in prices_nuevas if value != 0]
avg_nuevas = sum(prices_nuevas) / len(prices_nuevas)
print(avg_nuevas)